In [ ]:
import dotenv
dotenv.load_dotenv()

import sys
import os
import openai

openai.api_key = os.environ['OPENAI_KEY']


import json
import inspect

def safe_json_parse(json_str):
    try:
        return json.loads(json_str)
    except json.JSONDecodeError as e:
        # Handle the specific JSON parsing error
        print(json_str)
        return json_str


#gpt-3.5-turbo-1106
#gpt-4-1106-preview
class GPTtoolCaller:
    def __init__(self, model_name="gpt-3.5-turbo-1106", system_msg=None):
        self.model_name = model_name
        self.tools = {}
        self.system_msg = system_msg
        self.tool_definitions = []
        self.reset()
        
    def reset(self):
        self.messages=[]
        if not self.system_msg is None:
            self.messages.append({'role':'system','content':self.system_msg})

    def register_tool(self, func):
        self.tools[func.__name__] = func
        params = inspect.signature(func).parameters
        param_info = {
            "type": "object",
            "properties": {},
            "required": list(params.keys())
        }
        for param in params.values():
            # Example of how to fill the parameter details
            # You might need to adjust this depending on your specific needs
            param_info["properties"][param.name] = {
                "type": "string",  # or other type based on your tool
                "description": param.name  # or a more detailed description
            }
        self.tool_definitions.append({
            "type": "function",
            "function": {
                "name": func.__name__,
                "description": func.__doc__,
                "parameters": param_info
                },
        })
        return func

    def query(self, user_input):
        self.messages.append({"role": "user", "content": user_input})
        # First GPT-4 call to potentially trigger a tool call


        stopped = False
        while not stopped:
            response = openai.ChatCompletion.create(
                model=self.model_name,
                messages=self.messages,            
                tools=self.tool_definitions,
                tool_choice="auto",
                
            )

            print(response['choices']])
            for choice in response['choices']:
                #print(choice)
                message = choice.get('message')
                self.messages.append(message)
                tool_calls = message.get("tool_calls")
                if tool_calls:
                    for tool_call in tool_calls:
                        function_call = tool_call.get('function')
                        tool_name = function_call["name"]
                        arguments_json = function_call["arguments"]
                        print(tool_name,arguments_json)
                        # Use the safe JSON parsing method
                        parsed_arguments = safe_json_parse(arguments_json)

                        if "error" in parsed_arguments:
                            return parsed_arguments["error"]
                        elif tool_name in self.tools:
                            # Call the registered tool with the parsed arguments
                            tool_response = "NO REPONSE"
                            try:
                                tool_response = self.tools[tool_name](**parsed_arguments)
                            except Exception as e:
                                tool_response = "Error: " + str(e)

                            self.messages.append({
                                "tool_call_id": tool_call.id,
                                "role": "tool",
                                "name": tool_name,
                                "content": tool_response,
                            }) 
                if choice['finish_reason'] == "stop":
                    stopped = True


In [ ]:

gpt_caller = GPTtoolCaller(system_msg="""
Please take the users input.
TAlk to yourself consicely between tool calls. this is your memory.
do not write messages to the user, they will never see it.
use only the tools you have been given 

if you cannot answer then please call the tool UNKNOWN_command
""")

import tempfile

@gpt_caller.register_tool
def UNKNOWN_command(command):
    """This tool is called when the system does not know how to handle a command"""
    print(f"UNKNOWN COMMAND: {command}")


@gpt_caller.register_tool
def write_file(path, content):
    """This tool writes content to a file at the given path"""
    with open(path, "w") as f:
        f.write(content)
    return f"Wrote content to {path}"

@gpt_caller.register_tool
def read_file(path):
    """This tool reads the content of a file at the given path"""
    if not os.path.exists(path):
        return f"File does not exist at {path}"
    with open(path, "r") as f:
        return f.read()

@gpt_caller.register_tool
def create_dir(path):
    """This tool creates a directory at the given path"""
    os.mkdir(path)
    return f"Created directory at {path}"


@gpt_caller.register_tool
def copy_file(path, new_path):
    """This tool copies a file from the given path to the new path"""
    with open(path, "r") as f:
        content = f.read()
    with open(new_path, "w") as f:
        f.write(content)
    return f"Copied file from {path} to {new_path}"

@gpt_caller.register_tool
def append_file(path, content):
    """This tool appends content to a file at the given path"""
    with open(path, "a") as f:
        f.write(content)
    return f"Appended content to {path}"

@gpt_caller.register_tool
def replace_file(path, content):
    """This tool replaces the content of a file at the given path"""
    with open(path, "w") as f:
        f.write(content)
    return f"Replaced content of {path}"

@gpt_caller.register_tool
def apply_git_diff(path, patch):
    """This tool applies a git diff to a file at the given path, the diff should be in the unified format and will be applied with git apply"""
    tmp_path = "tmpfile"
    with open(tmp_path, "w") as f:
        f.write(patch)
    r = os.system(f"git apply {tmp_path}")
    return f"Applied git diff to {path} - {r}"


@gpt_caller.register_tool
def backup_file(path):
    """This tool copies a backup of a file from the given path"""
    with open(path, "r") as f:
        content = f.read()
    new_path = path+'.backup'
    with open(new_path, "w") as f:
        f.write(content)
    return f"backed up file from {path} to {new_path}"


@gpt_caller.register_tool
def report_code_quality(path, style, quality):
    """This tool reports the code quality of a file at the given path"""
    print(f"Code quality of {path} is {quality} with style {style}")
    return f"Code quality of {path} is {quality} with style {style}"

@gpt_caller.register_tool
def restore_file(path):
    """This tool restores a file from the backup at a given path"""
    with open(path, "r") as f:
        content = f.read()
    new_path = path.replace('.backup','')
    with open(new_path, "w") as f:
        f.write(content)
    return f"restored file from {path} to {new_path}"



@gpt_caller.register_tool
def create_dir(path):
    """This tool creates a directory at the given path"""
    if os.path.exists(path):
        return f"Directory already exists at {path}"
    os.mkdir(path)
    return f"Created directory at {path}"
    
@gpt_caller.register_tool   
def list_dir(path):
    """This tool lists the files and directories in the given path"""
    return "\n".join(os.listdir(path))


In [ ]:

def run_chat():
    print("Welcome to the GPT-4 chatbot, type 'exit' to quit")
    while True:
        user_input = input("You: ")
        if user_input == "exit":
            break
        print("User:",user_input)
        gpt_caller.query(user_input)
run_chat()

In [ ]:
gpt_caller.reset()
messages = gpt_caller.query("""
read the file at fake_bank_with_dummy_account.py and modify it to be a multi-account fast-api server service
after that: modify the homepage to have a list of accounts and a link to each account, and a link to create a new account
after that: modify the account page to have a list of transactions and a link to create a new transaction
make sure to add routes to the api to support these new features
""")


In [ ]:
gpt_caller.reset()
messages = gpt_caller.query(""" review code quality for fake_bank_with_dummy_account.py """)

In [ ]:
gpt_caller.reset()
messages = gpt_caller.query(""" review code quality for bad_code.py """)

In [ ]:
gpt_caller.reset()
messages = gpt_caller.query(""" rewrite the code inside  bad_code.py and save as better.py """)